# Load CSVs (one-to-many) to Azure Cognitive Search

In this Jupyter Notebook, we create and run steps to index a CSV file in which each row is an individual and independent record/document. Each row then becomes searchable in Azure Cognitive Search. 
The reference documentation can be found at [Indexing blobs and files to produce multiple search documents](https://learn.microsoft.com/en-us/azure/search/search-howto-index-one-to-many-blobs).

By default, an indexer will treat the contents of a blob or file as a single search document. If you want a more granular representation in a search index, you can set parsingMode values to create multiple search documents from one blob or file.

We are going to be using the same private Blob Storage account but a different container that has abstracts of 90k Medical publications about COVID-19 published in 2020-2022. This file is a subset of a much bigger dataset (770k articles) called CORD19 [HERE](https://github.com/allenai/cord19)

In [1]:
import os
import json
import requests
from dotenv import load_dotenv
load_dotenv("credentials.env")

# Name of the container in your Blob Storage Datasource ( in credentials.env)
BLOB_CONTAINER_NAME = "cord19"

In [2]:
# Define the names for the data source, index and indexer
datasource_name = "cogsrch-datasource-csv"
skillset_name = "cogsrch-skillset-csv"
index_name = "cogsrch-index-csv"
indexer_name = "cogsrch-indexer-csv"

In [3]:
# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_SEARCH_KEY']}
params = {'api-version': os.environ['AZURE_SEARCH_API_VERSION']}

## Create Data Source (Blob container with the Litcovid CSV data file)

In [4]:
# Create a data source

datasource_payload = {
    "name": datasource_name,
    "description": "Demo files to demonstrate cognitive search capabilities of one-to-many.",
    "type": "azureblob",
    "credentials": {
        "connectionString": os.environ['BLOB_CONNECTION_STRING']
    },
    "container": {
        "name": BLOB_CONTAINER_NAME
    }
}
r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/datasources/" + datasource_name,
                 data=json.dumps(datasource_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

204
True


## Inspect CSV file so we can understand the column types before creating the Index

In our private dataset we have place a smaller version of the original the metadata.csv file in the cord19 dataset. 
Let's see what the file looks like:

In [5]:
#Download the csv files to disk and inspect using pandas
import pandas as pd
remote_file_path = "https://datasetsgptsmartsearch.blob.core.windows.net/cord19/metadata.csv"

In [6]:
metadata = pd.read_csv(remote_file_path + os.environ['BLOB_SAS_TOKEN'])
print("No. of lines:",metadata.shape[0])

simple_schema = ['cord_uid', 'source_x', 'title', 'abstract', 'authors', 'url']

def make_clickable(address):
    '''Make the url clickable'''
    return '<a href="{0}">{0}</a>'.format(address)

def preview(text):
    '''Show only a preview of the text data.'''
    return text[:30] + '...'

format_ = {'title': preview, 'abstract': preview, 'authors': preview, 'url': make_clickable}

metadata[simple_schema].head().style.format(format_)

No. of lines: 90000


,cord_uid,source_x,title,abstract,authors,url
0,ug7v899j,PMC,Clinical features of culture-p...,OBJECTIVE: This retrospective ...,"Madani, Tariq A; Al-Ghamdi, Ai...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC35282/
1,02tnwd4m,PMC,Nitric oxide: a pro-inflammato...,Inflammatory diseases of the r...,"Vliet, Albert van der; Eiseric...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC59543/
2,ejv2xln0,PMC,Surfactant protein-D and pulmo...,Surfactant protein-D (SP-D) pa...,"Crouch, Erika C...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC59549/
3,2b73a28n,PMC,Role of endothelin-1 in lung d...,Endothelin-1 (ET-1) is a 21 am...,"Fagan, Karen A; McMurtry, Ivan...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC59574/
4,9785vg6d,PMC,Gene expression in epithelial ...,Respiratory syncytial virus (R...,"Domachowske, Joseph B; Bonvill...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC59580/


## Create Skillset - Text Splitter, Language Detection
We will use cognitive services enrichment for spliting the text of each content field into chunks (pages) and for language detection. We should always split the text since we don't know how big the content of each row might be.

In [7]:
# Create a skillset
skillset_payload = {
    "name": skillset_name,
    "description": "Splits Text and detect language",
    "skills":
    [
        {
            "@odata.type": "#Microsoft.Skills.Text.SplitSkill",
            "context": "/document",
            "textSplitMode": "pages",
            "maximumPageLength": 5000, # 5000 is default
            "defaultLanguageCode": "en",
            "inputs": [
                {
                    "name": "text",
                    "source": "/document/abstract"
                }
            ],
            "outputs": [
                {
                    "name": "textItems",
                    "targetName": "pages"
                }
            ]
        }
    ],
    "cognitiveServices": {
        "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
        "description": os.environ['COG_SERVICES_NAME'],
        "key": os.environ['COG_SERVICES_KEY']
    }
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/skillsets/" + skillset_name,
                 data=json.dumps(skillset_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

204
True


## Create the Index
In Azure Cognitive Search, both blob indexers and file indexers support a delimitedText parsing mode for CSV files that treats each line in the CSV as a separate search document.

### **Important**:
As you can see below and from the prior Notebook, there are 7 mandatory fields in the schema: `id, title, content, chunks, name, location, vectorized`. These fields must exist in any index that you create regardles of the datasource. Any additional fields are good to add so the engine can search relevant documents, however the mandatory fields must exist for all the code downstream work with no issues.

In [8]:
index_payload = {
    "name": index_name,  
    "fields": [
        {"name": "id", "type": "Edm.String", "key": "true", "searchable": "false", "retrievable": "true", "facetable": "false", "filterable": "false", "sortable": "false"},
        {"name": "title", "type": "Edm.String", "searchable": "true", "retrievable": "true", "facetable": "false", "filterable": "true", "sortable": "false"},
        {"name": "content", "type": "Edm.String", "searchable": "true", "retrievable": "true", "facetable": "false", "filterable": "false", "sortable": "false"},
        {"name": "chunks","type": "Collection(Edm.String)", "searchable": "false", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "name", "type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "location", "type": "Edm.String", "searchable": "false", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "vectorized", "type": "Edm.Boolean", "searchable": "false", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "authors", "type": "Edm.String", "searchable": "true", "retrievable": "true", "facetable": "false", "filterable": "false", "sortable": "false"},
        {"name": "metadata_storage_name", "type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "metadata_storage_path", "type":"Edm.String", "searchable": "false", "retrievable": "true", "filterable": "false", "sortable": "false"},
        {"name": "metadata_storage_last_modified", "type":"Edm.DateTimeOffset", "searchable": "false", "retrievable": "false", "filterable": "false", "sortable": "false"}
    ],
    "semantic": {
        "configurations": [
            {
                "name": "my-semantic-config",
                "prioritizedFields": {
                    "titleField": 
                        {
                            "fieldName": "title"
                        },
                    "prioritizedContentFields": [
                        { 
                            "fieldName":"content" 
                        }
                    ]
                }
            }
        ]
    }
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + index_name,
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

204
True


## Create and Run the Indexer - (runs the pipeline)
To create one-to-many indexers with CSV blobs, create or update an indexer definition with the delimitedText parsing mode

In [9]:
indexer_payload = {
    "name": indexer_name,
    "dataSourceName": datasource_name,
    "targetIndexName": index_name,
    "skillsetName": skillset_name,
    "schedule" : { "interval" : "PT2H"},
    "fieldMappings": [
        {
          "sourceFieldName" : "cord_uid",
          "targetFieldName" : "id"
        },
        {
          "sourceFieldName" : "abstract",
          "targetFieldName" : "content"
        },
        {
          "sourceFieldName" : "metadata_storage_name",
          "targetFieldName" : "name"
        },
        {
          "sourceFieldName" : "url",
          "targetFieldName" : "location"
        }
    ],
    "outputFieldMappings":
    [
        {
            "sourceFieldName": "/document/pages/*",
            "targetFieldName": "chunks"
        }
    ],
    "parameters" : { 
        "configuration" : { 
            "dataToExtract": "contentAndMetadata",
            "parsingMode" : "delimitedText", 
            "firstLineContainsHeaders" : True,
            "delimitedTextDelimiter": ","
        } 
    }
}
r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexers/" + indexer_name,
                 data=json.dumps(indexer_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

204
True


In [11]:
# Optionally, get indexer status to confirm that it's running
try:
    r = requests.get(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexers/" + indexer_name +
                     "/status", headers=headers, params=params)
    # pprint(json.dumps(r.json(), indent=1))
    print(r.status_code)
    print("Status:",r.json().get('lastResult').get('status'))
    print("Items Processed:",r.json().get('lastResult').get('itemsProcessed'))
    print(r.ok)
    
except Exception as e:
    print("Wait a few seconds until the process starts and run this cell again.")

200
Status: success
Items Processed: 0
True


**When the indexer finishes running we will have all 90,000 rows indexed properly as separate documents in our Search Engine!.**

## Creation of its corresponding vector-based index

In [12]:
index_payload = {
    "name": index_name + "-vector",
    "fields": [
        {"name": "id", "type": "Edm.String", "key": "true", "filterable": "true" },
        {"name": "title","type": "Edm.String","searchable": "true","retrievable": "true"},
        {"name": "chunk","type": "Edm.String","searchable": "true","retrievable": "true"},
        {"name": "chunkVector","type": "Collection(Edm.Single)","searchable": "true","retrievable": "true","dimensions": 1536,"vectorSearchConfiguration": "vectorConfig"},
        {"name": "name", "type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "location", "type": "Edm.String", "searchable": "false", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},

    ],
    "vectorSearch": {
        "algorithmConfigurations": [
            {
                "name": "vectorConfig",
                "kind": "hnsw"
            }
        ]
    },
    "semantic": {
        "configurations": [
            {
                "name": "my-semantic-config",
                "prioritizedFields": {
                    "titleField": {
                        "fieldName": "title"
                    },
                    "prioritizedContentFields": [
                        {
                            "fieldName": "chunk"
                        }
                    ],
                    "prioritizedKeywordsFields": []
                }
            }
        ]
    }
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + index_name + "-vector",
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

204
True


Bad pipe message: %s [b'.K\xd5\x19\xfc\x8d~M\xe3Y\xc2\xe3\x1bxW', b"\xe4 M\x1fv\xbf3\x01\xf7p\xc1\xe5'\x81!\xbf\xba\xe0\xf2\xd4\xd3\x92\x1c\x8f7\x8a\x8a6X\\N5NE\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x00+\x00\x03\x02"]
Bad pipe message: %s [b'\x00-']
Bad pipe message: %s [b"\xb4\x0e~8\xaa\xdc\x9by\xeb\x10\xb6b\xdd1\xa1\x85\x91\xed\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=

# Reference

- https://learn.microsoft.com/en-us/azure/search/search-howto-index-csv-blobs
- https://learn.microsoft.com/en-us/azure/search/knowledge-store-create-rest



# NEXT
Now that we have two separate text-based indexes loaded with two different types of information and its correspongind vector-based indexes, In the next notebook 3, we will do a Multi-Index query, sort the results based on the reranker semantic score of Azure Search, and then use OpenAI to understand this results and give the best answer possible